# 导入用到的库和构造测试数据表
假设users记录了用户的user_id,name和age。

trade记录了用户的交易日期date和交易金额amount，一个用户有多条交易记录。

In [1]:
from sqlalchemy import Table, MetaData, Column, Integer, String,select,func

metadata = MetaData()
users = Table(
    'users',
    metadata,
    Column('user_id',Integer),
    Column('name',String),
    Column('age',String)    
)

trades = Table(
    'trades',
    metadata,
    Column('user_id',Integer),
    Column('date',String),
    Column('amount',Integer)    
)

In [30]:
def create_sql(user_id_list, attr_list):
    # 清完成这个函数。作为练习，这个函数中可以引用全局变量的方式使用users和trades,请按照要求补全代码
    
    attr_dict = {
        'name':{
            'table':users,
            'column':'name',
            'func':None,
        },
        'age':{
            'table':users,
            'column':'age',
            'func':None,
        },
        'first_date':{
            'table':trades,
            'column':'date',
            'func':func.min,
        },
        'amount_sum':{
            'table':trades,
            'column':'amount',
            'func':func.sum,
        },
    }

    import pandas as pd
    df = pd.DataFrame.from_dict(attr_dict, orient='index')

    table_list = df.loc[attr_list]['table'].drop_duplicates().tolist()

    s = None

    for table in table_list:
        s1 = None
        s2 = None

        none_func_attr_list = list(set(df[pd.isnull(df['func'])].loc[df['table'] == table].index) & set(attr_list))
        func_attr_list = list(set(df[pd.notnull(df['func'])].loc[df['table'] == table].index) & set(attr_list))

        if none_func_attr_list:
            s1 = select(
                [
                    table.c.user_id,
                    *[
                        table.c[df.loc[attr]['column']] 
                        for attr in none_func_attr_list
                    ],
                ]
            ).where(
                table.c.user_id.in_(user_id_list)
            )

        if func_attr_list:
            s2 = select(
                [
                    table.c.user_id,
                    *[
                        df.loc[attr]['func'](table.c[df.loc[attr]['column']]).label(attr)
                        for attr in func_attr_list
                    ],
                ]
            ).where(
                table.c.user_id.in_(user_id_list)
            ).group_by(
                table.c.user_id
            )

        if s is None:
            if s1 is not None:
                s = s1
            elif s2 is not None:
                s = s2
        else:
            if s1 is not None:
                s = select(
                    [
                        s.c.user_id,
                        *[s.c[col] for col in s.c.keys() if col != 'user_id'],
                        *[s1.c[col] for col in s1.c.keys() if col != 'user_id']
                    ]
                ).select_from(
                    s.join(
                        s1,
                        s.c.user_id == s1.c.user_id
                    )
                )

            if s2 is not None:
                s = select(
                    [
                        s2.c.user_id,
                        *[s.c[col] for col in s.c.keys() if col != 'user_id'],
                        *[s2.c[col] for col in s2.c.keys() if col != 'user_id']
                    ]
                ).select_from(
                    s.join(
                        s2,
                        s.c.user_id == s2.c.user_id
                    )
                )
    
    return s
    
    # Complete the code
    # pass

# create_sql的要求
函数接受两个参数
* user_id_list表示查询id的范围
* attr_list表示查询的属性名组成的list,可以是'name','age','first_date','amount_sum'中某几个的组合
    * amount_sum是总交易金额
    * first_date是第一次交易日期

函数返回的内容是能够实现查询逻辑的sqlalchmy object,要求是能够根据查询的属性生成对应的SQL逻辑。

例一

In [ ]:
create_sql([1,2,3], ['name'])

In [31]:
print(create_sql([1,2,3], ['name']))

SELECT users.user_id, users.name 
FROM users 
WHERE users.user_id IN (:user_id_1, :user_id_2, :user_id_3)


In [32]:
print(create_sql([1,2,3], ['first_date']))

SELECT trades.user_id, min(trades.date) AS first_date 
FROM trades 
WHERE trades.user_id IN (:user_id_1, :user_id_2, :user_id_3) GROUP BY trades.user_id


那么返回结果应该等同于s

In [3]:
s = select(
    [
        users.c['name']    
    ]
).where(
    users.c['user_id'].in_([1,2,3])
)

In [4]:
print(s)

SELECT users.name 
FROM users 
WHERE users.user_id IN (:user_id_1, :user_id_2, :user_id_3)


例二

In [ ]:
create_sql([1,2,3], ['name','age','first_date'])

In [33]:
print(create_sql([1,2,3], ['name','age','first_date']))

SELECT user_id, name, age, first_date 
FROM (SELECT users.user_id AS user_id, users.name AS name, users.age AS age 
FROM users 
WHERE users.user_id IN (:user_id_1, :user_id_2, :user_id_3)) JOIN (SELECT trades.user_id, min(trades.date) AS first_date 
FROM trades 
WHERE trades.user_id IN (:user_id_4, :user_id_5, :user_id_6) GROUP BY trades.user_id) ON user_id = user_id


那么返回的结果应该等同于s

In [7]:
s1 = select(
    [
        users.c.user_id,
        users.c['name'],    
        users.c['age'],        
    ]
).where(
    users.c['user_id'].in_([1,2,3])
)

s2 = select(
    [
        trades.c.user_id,
        func.min(trades.c['date']).label('first_date')        
    ]
).where(
    trades.c['user_id'].in_([1,2,3])
).group_by(
    trades.c['user_id']
)


s = select(
    [
        s1.c.user_id,
        s1.c['name'],
        s1.c['age'],
        s2.c['first_date']        
    ]
).select_from(
    s1.join(s2,
        s1.c.user_id == s2.c.user_id            
    )
)

In [8]:
print(s)

SELECT user_id, name, age, first_date 
FROM (SELECT users.user_id AS user_id, users.name AS name, users.age AS age 
FROM users 
WHERE users.user_id IN (:user_id_1, :user_id_2, :user_id_3)) JOIN (SELECT trades.user_id, min(trades.date) AS first_date 
FROM trades 
WHERE trades.user_id IN (:user_id_4, :user_id_5, :user_id_6) GROUP BY trades.user_id) ON user_id = user_id
